# Adaptive PDE discretizations on Cartesian grids
## Volume : Non-divergence form PDEs
## Part : One space dimension
## Chapter : Optimal transport non-linearities and boundary conditions

The PDE formulation of the optimal transport problem reads as follows:
$$
    f(x)-g(\nabla u(x))\det(\nabla^2 u(x)) = 0,
$$
subject to the constraint
$$
    u \quad \text{convex},
$$
and the boundary condition
$$
    \nabla u(X) = Y,
$$
where $X$ is the source domain, and $Y$ is the target domain. The parameters of the PDE are the densities $f$ and $g$, which are non-negative and subject to the compatibility condition
$$
    \int_X f(x) dx = \int_Y g(y) dy.
$$

This mathematical problem comes with a number of challenges, of both theoretical and numerical nature, including the:
1. Full non-linearity of the second order operator.
2. Constraint of convexity
3. Unspecified non-linearity of the first order operator.
4. Lack of a comparison principle.

Challenges 1. and 2. are related with the Monge-Ampere operator $\det(\nabla^2 u)$. They are discussed  the notebook 
[C.I Monge-Ampere equations](http://nbviewer.jupyter.org/urls/rawgithub.com/Mirebeau/AdaptiveGridDiscretizations/master/Notebooks/MongeAmpere.ipynb)

This notebook is devoted to challenges 3. and 4., in a simplified one dimensional setting.
In practice, 3. is mostly an implementation problem. Substantial theoretical work has been required to address 4. however. It is based on the observation that the second boundary condition could be weakened into 
$$
    \nabla u(X) \subset Y,
$$
and that equality must hold for the solution due to the compatibility condition relating the problem parameter densities.

**Note on one dimensional optimal transport.**
It is well known that one dimensional optimal transport (with a convex cost function) is nothing else than a sorting problem. As such it can be solved extremely efficiently, at a quasi-linear cost, and is considered a solved algorithmic problem.
The interest of numerical techniques presented in this notebook lies is their possible extension to higher dimensions.

**References**

* Benamou, J.-D., & Duval, V. (2017). Minimal convex extensions and finite difference discretisation of the quadratic Monge--Kantorovich problem. European Journal of Applied Mathematics, 1–38.

## The one-dimensional problem

In dimension one, the Monge-Ampere operator reduces to the usual laplacian operator, and the related difficulties 1. and 2. disappear. We are left with the equation
$$
    f(x) - g(u'(x)) u''(x) = 0,
$$
subject to the boundary conditions
$$
    u'(x_*) =  y_*, \quad u'(x^*) = y^*,
$$
where $X=[x_*,x^*]$ and $Y = [y_*,y^*]$ are the source and target intervals. Again the problem parameters are subject to the compatibility conditions
$$
    \int_{x_*}^{x^*} f(x) dx = \int_{y_*}^{y^*} g(y) dy,
$$
and this allows to weaken the boundary conditions into 
$$
    u'(x_*) \geq  y_*, \quad u'(x^*) \leq y^*.
$$

[**Summary**](Summary.ipynb) of volume Non-Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Second boundary conditions](#1.-Second-boundary-conditions)
  * [2. Non-linearity in the gradient](#2.-Non-linearity-in-the-gradient)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; TocTools.displayTOC('OTBoundary1D','NonDiv')

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd import AutomaticDifferentiation as ad

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import time

Some utility functions

In [10]:
newton_root = ad.Optimization.newton_root

## 1. Second boundary conditions

We further simplify the problem and assume $g \equiv 1$, to study the weakened *second boundary conditions* that we rewrite as 
$$
    y_* - u'(x_*) \leq 0, \quad u'(x^*) - y^*\leq 0,
$$
at the endponts of the interval $[x_*,x^*]$ of definition of the problem solution $u$.
Recall that the PDE solved is 
$$
    f(x) - u''(x) = 0,
$$
and that we have 
$$
    \int_{x_*}^{x^*} f(x) dx = y^*-y_*.
$$

**Explicit solution.**
The above problem admits a one dimensional, family of explicit solutions. Namely
$$
u(x) = \alpha + r x + v(x),
$$
where $v$ is a second primitive of $u$, and the constant $r$ is adequately chosen to match the boundary conditions.
The other constant, denoted $\alpha$, is *arbitrary*.

### Montone discretization

We may define a monotone (degenerate elliptic) discretization of the boundary conditions, using upwind finite differences as follows
$$
    \max\{0,y_* - \frac{u(x_*+h)-u(x_*)} h\} = 0, \quad \max\{0,\frac{u(x^*)-u(x^*-h)} h - y^*\}= 0.
$$
We also use the standard discretization of the laplacian operator in the interior.

The resulting discretization obeys the assumptions of a degenerate elliptic (monotone) scheme, since it takes the form
$$
    F u(x) := F(x,u(x),(u(x)-u(x'))_{x' \in X} ),
$$
where $F$ is a non-decreasing function of its second and third arguments.

### Non-uniqueness and lack of a comparision principle

The continuous problems admits, as mentioned above, a one dimensional family of solutions. The discrete problem, on the other hand, admits either no solution, or a large and typically multi-dimensional family of solutions.

These indeterminacies reflect a failure of the comparison principle, due to the impossibility to perturb a given solution into a strict sub-solution.

In [6]:
def Scheme(u,f,bc,h):
    u,f,bc = ad.common_cast(u,f,bc) # Safe AD scalars
    residue = f-fd.Diff2(u,(1,),h)
    residue[0]  = np.maximum(0.,bc[0] - (u[1]-u[0])/h)
    residue[-1] = np.maximum(0.,(u[-1]-u[-2])/h - bc[1])
    return residue

In [7]:
X = np.linspace(-1,1,101,endpoint=True)
gridScale = X[1]-X[0]
f = 1.+0.9*np.sin(2.*np.pi*X)
bc = np.array((-1.,1.))

guess = np.zeros(X.shape)

The discretized problem is linear. However, the jacobian matrix $J$ is rank deficient and non-invertible.
Fortunately, thanks to the scheme monotony, the matrix 
$$
    J+\epsilon \mathrm{Id} 
$$
is invertible for every $\epsilon>0$.
We use an iterative method, featuring such a relaxation, so solve the problem.

In order to illustrate the dichotomy between *no solution* and *infinitely may solutions, we perturb a bit the boundary conditions.

**Note on convergence (No solution).**
The discretized problem addressed here has *no solution*. As a result, the Newton solver residues do not tend to zero (or numerical precision). This is expected behavior.

In [13]:
bc2 = np.array((-1,1-2*gridScale))
params = (f,bc2,gridScale)
relax = ad.Sparse.identity(X.shape)*gridScale
stop = ad.Optimization.stop_default(raise_on_abort=False)
solution = newton_root(Scheme,guess,params,relax=relax,stop=stop)

Iteration: 1  Residue norm: 0.12636527953248167
Iteration: 2  Residue norm: 0.009998009013359899
Iteration: 3  Residue norm: 0.005066525567450353
Iteration: 4  Residue norm: 0.005025995705963959
Iteration: 5  Residue norm: 0.00502514748832894
Iteration: 6  Residue norm: 0.005025126195172813
Iteration: 8  Residue norm: 0.005025125631078264
Iteration: 10  Residue norm: 0.005025125631030969
Iteration: 12  Residue norm: 0.005025125631031191
Iteration: 14  Residue norm: 0.005025125632292626
Iteration: 16  Residue norm: 0.005025125633520866
Iteration: 20  Residue norm: 0.0050251256338067485
Iteration: 24  Residue norm: 0.005025125634669059
Iteration: 28  Residue norm: 0.005025125640088168
Iteration: 32  Residue norm: 0.005025125636259453
Iteration: 36  Residue norm: 0.0050251256377495945
Iteration: 40  Residue norm: 0.005025125639784633
Iteration: 44  Residue norm: 0.005025125640482742
Iteration: 48  Residue norm: 0.005025125640482742
Iteration: 50  Residue norm: 0.005025125639711137
Max ite

In [14]:
plt.title("Closest solution to 1-d optimal transport problem")
plt.plot(X,solution);

We next perturb a bit the boundary conditions, ensuring, this time, that the discretized problem has *an infinity of solutions*.

**Note on convergence (degenerate solutions).**
The discretized problem addressed here has a two dimensional non-linear family of solutions. It is also non-differentiable. As a result, the Newton solve takes more steps to reach numerical precision than what could be expected for problems this size.

In [16]:
bc3 = np.array((-1,1+2*gridScale))
params = (f,bc3,gridScale)
relax = ad.Sparse.identity(X.shape) * gridScale
solution = newton_root(Scheme,guess,params,relax=relax)

Iteration: 1  Residue norm: 0.12636527953248167
Iteration: 2  Residue norm: 0.0049899647157736204
Iteration: 3  Residue norm: 0.0003108032158998242
Iteration: 4  Residue norm: 1.9903133235965598e-05
Iteration: 5  Residue norm: 1.2774458824083013e-06
Iteration: 6  Residue norm: 8.200718704109988e-08
Iteration: 7  Residue norm: 5.264649827196877e-09
Target residue reached. Terminating.


## 2. Non-linearity in the gradient 

We solve the PDE 
$$
f(x)/g(u'(x)) - u''(x)=0
$$

In [17]:
def SchemeGradient(u,f,g,bc,h):
    u,f,bc = ad.common_cast(u,f,bc) # Safe array scalars
    residue = f/g(fd.DiffCentered(u,(1,),h))-fd.Diff2(u,(1,),h)
    residue[0]  = np.maximum(0.,bc[0] - (u[1]-u[0])/h)
    residue[-1] = np.maximum(0.,(u[-1]-u[-2])/h - bc[1])
    return residue

In [18]:
X = np.linspace(-1,1,101,endpoint=True)
gridScale = X[1]-X[0]
f = 1.+0.9*np.sin(2.*np.pi*X)
bc = np.array((-1.,1.))

g = 1.+0.5*np.cos(2.*np.pi*X)
f*=g.sum()/f.sum()
g_interp=fd.UniformGridInterpolator1D(X,g)

guess = np.zeros(X.shape)

In [19]:
params = (f,g_interp,bc,gridScale)
relax = ad.Sparse.identity(X.shape) * gridScale
solution = newton_root(SchemeGradient,X,params,relax=relax)

Iteration: 1  Residue norm: 2.276164965291264
Iteration: 2  Residue norm: 6.9793294237230175
Iteration: 3  Residue norm: 8.62442150407455
Iteration: 4  Residue norm: 9.030547460925924
Iteration: 5  Residue norm: 2.1593586930666406
Iteration: 6  Residue norm: 9.625961543889987
Iteration: 8  Residue norm: 0.011620928558317534
Iteration: 10  Residue norm: 0.0031334056758374196
Iteration: 12  Residue norm: 7.999409468162355e-08
Iteration: 13  Residue norm: 5.972761174533048e-09
Target residue reached. Terminating.


Check the solution.

In [20]:
def g_exact(X): return  1.+0.5*np.cos(2.*np.pi*X)
g_exact(fd.DiffCentered(solution,(1,),gridScale))*fd.Diff2(solution,(1,),gridScale) - f

array([            nan,  6.87742984e-04,  4.25196993e-04,  6.73178903e-05,
        1.32363623e-04,  4.45170479e-05,  3.27466290e-04,  7.37163922e-04,
        7.38002938e-04,  1.51154859e-04,  3.57739169e-04,  4.42063796e-06,
       -5.29404330e-04, -6.49207982e-04, -2.02727836e-03, -8.13380573e-04,
       -3.11914280e-03, -6.59402656e-04, -1.30583764e-03, -4.67608298e-04,
       -1.92586905e-04,  4.54341273e-05,  2.70857291e-04,  3.77270942e-04,
        4.26255690e-04,  3.84348262e-04,  6.73967576e-05,  3.50017317e-04,
        2.35552702e-04,  2.84909032e-04,  2.42766122e-05,  1.98972912e-04,
        1.76881340e-04,  1.02023536e-04,  3.46008538e-05,  9.93660509e-06,
        3.01323038e-05,  3.95244907e-05,  4.91219534e-05,  6.44654555e-05,
        8.73718644e-05,  1.13774460e-04,  1.26276361e-04,  7.92677604e-05,
        1.02317288e-04,  2.91498166e-04,  2.61703919e-04,  2.59976561e-04,
        4.49189841e-04,  4.13061030e-04,  3.15628354e-04,  7.20788425e-04,
        7.32809426e-04,  